<a href="https://colab.research.google.com/github/KenyanGeek/Retail-Data-Science-and-Machine-Learning/blob/master/7d_Earnings_Predictions_Fast_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting 7D Earnings using Tabular Data

In [0]:
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.

In [66]:
from google.colab import files
uploaded = files.upload()


Saving data.csv to data.csv


In [67]:
import io


df = pd.read_csv(io.BytesIO(uploaded['data.csv']))

df

,day,date,netprofit,expense,bonus,grossprofit,year,month,daynumber,yearquarter,monthcorner,weekend,electionyear,profitable,kenyaholiday
0,Monday,2015-09-07,2050,100,0,1950,2015,9,7,3RD QUARTER,BOM,False,False,True,notholiday
1,Tuesday,2015-09-08,1750,50,0,1700,2015,9,8,3RD QUARTER,BOM,False,False,True,notholiday
2,Wednesday,2015-09-09,700,0,0,700,2015,9,9,3RD QUARTER,BOM,False,False,True,notholiday
3,Thursday,2015-09-10,1100,0,0,1100,2015,9,10,3RD QUARTER,BOM,False,False,True,notholiday
4,Friday,2015-09-11,1100,0,0,1100,2015,9,11,3RD QUARTER,BOM,False,False,True,notholiday
5,Saturday,2015-09-12,750,0,0,750,2015,9,12,3RD QUARTER,BOM,True,False,True,notholiday
6,Monday,2015-09-14,1250,50,0,1200,2015,9,14,3RD QUARTER,BOM,False,False,True,notholiday
7,Tuesday,2015-09-15,2150,100,0,2050,2015,9,15,3RD QUARTER,MOM,False,False,True,notholiday
8,Wednesday,2015-09-16,1700,500,0,1200,2015,9,16,3RD QUARTER,MOM,False,False,True,notholiday
9,Friday,2015-09-18,1550,50,0,1500,2015,9,18,3RD QUARTER,MOM,False,False,True,notholiday


In [0]:
dep_var = 'netprofit'
cat_names = ['day', 'date','month','monthcorner','daynumber','yearquarter','weekend','electionyear','year']
cont_names = ['grossprofit']
procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df.iloc[1000:1287].copy(), cat_names=cat_names, cont_names=cont_names)

In [102]:
data = (TabularList.from_df(df, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())


/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:525: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
3600, 3900, 3300, 2800, 3150
  if getattr(ds, 'warn', False): warn(ds.warn)


In [103]:
data.show_batch(rows=10)

day,date,month,monthcorner,daynumber,yearquarter,weekend,electionyear,year,grossprofit,target
Tuesday,2016-05-31,5,#na#,31,2ND QUARTER,False,False,2016,-0.0597,1330
Thursday,2016-09-29,9,EOM,29,3RD QUARTER,False,False,2016,-1.2404,650
Tuesday,2018-02-06,2,BOM,6,1ST QUARTER,False,False,2018,-0.8862,900
Saturday,2017-12-02,12,BOM,2,4TH QUARTER,True,True,2017,0.1765,1450
Thursday,2016-02-25,2,EOM,25,1ST QUARTER,False,False,2016,-1.1224,800
Wednesday,2016-10-05,10,BOM,5,3RD QUARTER,False,False,2016,0.5307,1650
Friday,2015-10-30,10,EOM,30,3RD QUARTER,False,False,2015,-0.4139,1000
Sunday,2017-12-17,12,MOM,17,4TH QUARTER,True,True,2017,-1.0043,850
Tuesday,2018-05-01,5,BOM,1,2ND QUARTER,False,False,2018,-0.2958,1150
Thursday,2017-04-13,4,BOM,13,1ST QUARTER,False,True,2017,-1.1224,800


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [105]:
learn.fit(50, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.373788,3.088931,0.250000,00:00
1,2.682414,3.271575,0.067708,00:00
2,2.123916,3.045070,0.052083,00:00
3,1.619437,2.536251,0.114583,00:00
4,1.223432,2.575028,0.234375,00:00
5,0.917051,2.642525,0.234375,00:00
6,0.689357,2.703554,0.281250,00:00
7,0.516462,2.634873,0.265625,00:00
8,0.388716,2.716580,0.255208,00:00
9,0.293616,2.725628,0.239583,00:00


## Inference

In [0]:
row = df.iloc[1060]

In [119]:
df.iloc[1060]

day                 Tuesday
date             2019-03-05
netprofit              1450
expense                  50
bonus                   100
grossprofit            1300
year                   2019
month                     3
daynumber                 5
yearquarter     1ST QUARTER
monthcorner             BOM
weekend               False
electionyear          False
profitable             True
kenyaholiday     notholiday
Name: 1060, dtype: object

In [120]:
learn.predict(row)

(Category 1450,
 tensor(28),
 tensor([2.2208e-08, 1.2967e-08, 1.4879e-08, 3.5921e-09, 2.8128e-10, 3.4616e-10,
         3.1378e-08, 1.9713e-09, 8.6156e-08, 2.9180e-09, 2.2465e-08, 4.5473e-06,
         4.0385e-08, 7.5502e-08, 1.0868e-06, 3.7366e-06, 9.1149e-09, 5.0809e-06,
         6.3764e-07, 3.5872e-06, 7.3377e-06, 9.5218e-09, 1.5379e-08, 1.4722e-08,
         7.8942e-06, 2.4939e-08, 4.0894e-07, 6.3150e-06, 9.9990e-01, 3.6484e-07,
         9.7379e-08, 1.1753e-06, 3.9892e-07, 1.0005e-07, 7.7690e-07, 1.4405e-05,
         6.1537e-07, 6.0155e-06, 6.8293e-07, 4.9029e-06, 2.1583e-07, 4.3268e-08,
         4.4711e-07, 2.3750e-08, 3.0065e-06, 1.8631e-06, 2.7581e-07, 4.0723e-06,
         1.9298e-06, 1.0945e-06, 8.5212e-07, 9.2825e-06, 7.9858e-07, 1.1619e-08,
         1.4623e-08, 2.7486e-08, 1.2058e-08, 3.9280e-06, 4.0006e-08, 9.0201e-10,
         3.4646e-08, 3.8746e-07, 2.3957e-07, 1.0029e-07, 9.9897e-07, 9.4470e-09,
         1.4981e-08, 2.9928e-07, 6.7325e-09, 1.0596e-07, 9.9603e-08, 6.0320e-08,